In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
!pip install torch

In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
class MNISTData(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.data[idx]

In [5]:
class MNISTTrainData(MNISTData):
    def __getitem__(self, idx):
        X = self.data[idx][1:] / 255.0    # Normalisation
        y = self.data[idx][0]
        return X,y

In [6]:
class MNISTTestData(MNISTData):
    def __getitem__(self, idx):
        X = self.data[idx] / 255.0     # Normalisation
        return X

In [7]:
import pandas as pd

In [8]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [9]:
print(train_data.shape)
print(test_data.shape)

(42000, 785)
(28000, 784)


In [10]:
train_data = MNISTTrainData(train_data.values)
test_data = MNISTTestData(test_data.values)

In [11]:
batch_size = 512
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=batch_size, shuffle=False)

In [12]:
import torch
import torch.nn as nn

In [24]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(28*28, 16),
            nn.Linear(16, 16),
            nn.Linear(16, 10),
        )
        
    def forward(self, x):
        output = self.mlp(x)
        return output

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
learning_rate = 0.01

In [36]:
model = MLP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [37]:
print(model)

MLP(
  (mlp): Sequential(
    (0): Linear(in_features=784, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=16, bias=True)
    (2): Linear(in_features=16, out_features=10, bias=True)
  )
)


In [38]:
epochs = 20

model.train()

for epoch in range(epochs):
    losses = []
    
    for batch_num, input_data in enumerate(train_loader):
        optimizer.zero_grad()
        
        x,y = input_data
        x = x.to(device).float()
        y = y.to(device)
        
        output = model(x)
        
        loss = criterion(output, y)
        loss.backward()
        losses.append(loss.item())
        
        optimizer.step()
        
        if batch_num % 40 == 0:
            print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
    print('Epoch %d | Loss %6.2f' % (epoch, sum(losses)/len(losses)))

	Epoch 0 | Batch 0 | Loss   2.30
	Epoch 0 | Batch 40 | Loss   0.45
	Epoch 0 | Batch 80 | Loss   0.36
Epoch 0 | Loss   0.53
	Epoch 1 | Batch 0 | Loss   0.32
	Epoch 1 | Batch 40 | Loss   0.33
	Epoch 1 | Batch 80 | Loss   0.33
Epoch 1 | Loss   0.30
	Epoch 2 | Batch 0 | Loss   0.35
	Epoch 2 | Batch 40 | Loss   0.32
	Epoch 2 | Batch 80 | Loss   0.29
Epoch 2 | Loss   0.29
	Epoch 3 | Batch 0 | Loss   0.30
	Epoch 3 | Batch 40 | Loss   0.28
	Epoch 3 | Batch 80 | Loss   0.20
Epoch 3 | Loss   0.28
	Epoch 4 | Batch 0 | Loss   0.28
	Epoch 4 | Batch 40 | Loss   0.24
	Epoch 4 | Batch 80 | Loss   0.23
Epoch 4 | Loss   0.27
	Epoch 5 | Batch 0 | Loss   0.28
	Epoch 5 | Batch 40 | Loss   0.21
	Epoch 5 | Batch 80 | Loss   0.27
Epoch 5 | Loss   0.27
	Epoch 6 | Batch 0 | Loss   0.24
	Epoch 6 | Batch 40 | Loss   0.25
	Epoch 6 | Batch 80 | Loss   0.19
Epoch 6 | Loss   0.26
	Epoch 7 | Batch 0 | Loss   0.33
	Epoch 7 | Batch 40 | Loss   0.28
	Epoch 7 | Batch 80 | Loss   0.31
Epoch 7 | Loss   0.26
	Epoch 8 | Batch

In [39]:
import csv
model.eval()

results = []

with torch.no_grad():
    for batch_number, x in enumerate(test_loader):
        x = x.to(device).float()

        output = model(x).argmax(dim=1)
        for out in output:
            results.append(out.item())

In [31]:
len(results)

28000

In [40]:
submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [41]:
submission['Label'] = results

In [42]:
submission.to_csv("/kaggle/working/submission.csv", index = False)